In [ ]:
import config
from phemex import *
import json
import pandas as pd
from datetime import datetime
import nest_asyncio
import time
import ta
import numpy as np
import ccxt
import talib
from talib import stream

nest_asyncio.apply()
import asyncio
import websockets
clients = {}
# Create an instance of the Phemex exchange
exchange = initExchange()
symbol = config.TRADE_SYMBOL
trades = pd.DataFrame(columns=['timestamp', 'side', 'priceEp', 'volume'])
tradesPrices = []
ohlcvData = pd.DataFrame(
    columns=['open', 'high', 'low', 'close', 'volume', 'timestamp'])
klineData = pd.DataFrame(
    columns=['timestamp', 'interval', 'lastCloseEp', 'openEp', 'highEp', 'lowEp', 'closeEp', 'volume', 'turnoverEv'])
tick_size = 0.5
quantity = 100
stop_distance = 10
stop_price = 0

# Set up exchange client
exchange = initExchange()

In [ ]:
async def placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False):  
    global stop_price
    
    stop_distance = 30 * 10000  
    try:
        existing_orders = exchange.fetch_open_orders(symbol=config.TRADE_SYMBOL)  
    except Exception as e:
        print('Error {}'.format(e))
        pass 
    
    limitFound = False 
    stoplossFound = False 
    stoploss_order = None      
    for order in existing_orders:
        # print('Existing orders: type={} side={}'.format(order['info']['orderType'],order['info']['side']))
        if order['info']['orderType'] == 'Limit' and not limitFound:
            side = order['info']['side']
            # print(f'Stopploss order: {stoploss_order}')
            limitFound = True  
        if order['info']['orderType'] == 'Stop' and not stoplossFound:
            side = order['info']['side']
            stoploss_order = order
            # print(f'Stopploss order: {stoploss_order}')
            stoplossFound = True  
    # Check if .MBTC ticker is above last price
    if int(lastMbtcTick) > int(lastClose):
        print(f'-= MBTC {lastMbtcTick} above Last close {lastClose} =-')   
        # print(f'difference {difference} > 2')
        # Check if there is a short position
        if position['side'] == 'Sell' and stoplossFound == False and addStoploss == True: 
            # Set stop loss slightly below liquidation price
            if stop_price == 0:
                stop_price = int(position['liquidationPrice'] - stop_distance)
            else:
                stop_price = stop_price - stop_distance
            # print(f'stop price {stop_price}')                  
            # Place the stop-loss and limit orders
            params = {
                    "orderType": "Stop",
                    "timeInForce": "ImmediateOrCancel",
                    "trigger": "ByMarkPrice",
                    "stopDirection": "Falling",
                    "stopPxEp": stop_price,
                    "stopPx": stop_price * 0.0001,
                    # "stopLossEp": stop_price,
                    "execInst": "CloseOnTrigger",
            }   
            # print('1')         
            stoploss_order = exchange.create_order(symbol=config.TRADE_SYMBOL, type='stop', side='buy', price=position['liquidationPrice'] * 0.0001, amount=quantity, params=params)     
            # stoploss_order = await place_stop_loss_order(config.TRADE_SYMBOL, 'buy', quantity, stop_price)           
            print('Stoploss created at:', stop_price)   
            stoplossFound = True                        
        # elif position['side'] == 'Sell' and stoplossFound == True:   
        #     if stop_price == 0:
        #         stop_price = int(position['liquidationPrice'] - stop_distance)
        #     else:
        #         stop_price = stop_price - stop_distance
        #     print(f'Edit stop order')
        #     price = position['liquidationPrice'] * 0.0001
        #     params = {'stopLossEp': stop_price}
        #     # print('4')
        #     if difference in range(0,2):              
        #         time.sleep(3)
   
        #     exchange.edit_order(id=stoploss_order['id'], symbol=config.TRADE_SYMBOL, price=price,params=params)                     
        # Update the stop-loss and limit orders
        elif position['side'] == 'Buy' and stoplossFound == True and addStoploss == False:
            # Remove stop loss
            # print('5')
            exchange.cancel_order(stoploss_order['id'], symbol)                  
            print('Remove Stoploss')
            cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
            # print('All stoploss orders removed',cancelResponse)
            stoploss_order = None                                             
    # Check if .MBTC ticker is below last price
    elif int(lastMbtcTick) < int(lastClose):
        print(f'-= MBTC {lastMbtcTick} below Last close {lastClose} =-')  
        # print(f'difference {difference} > 2')
        # Check if there is a long position
        if position['side'] == 'Buy' and stoplossFound == False and addStoploss == True:
            # Set stop loss slightly above liquidation price
            if stop_price == 0:
                stop_price = int(position['liquidationPrice'] + stop_distance)
            else:
                stop_price = stop_price + stop_distance
            params = {
                    "orderType": "Stop",
                    "timeInForce": "ImmediateOrCancel",
                    "trigger": "ByMarkPrice",
                    "stopDirection": "Rising",
                    "stopPxEp": stop_price,
                    "stopPx": stop_price * 0.0001,
                    # "stopLossEp": stop_price,
                    "execInst": "CloseOnTrigger",
            }
            price = position['liquidationPrice']
            # print('2')
            # print(f'price: {price} stopLoss: {stop_price}')
            stoploss_order = exchange.create_order(symbol=config.TRADE_SYMBOL, type='stop', side='sell', price=price, amount=quantity, params=params)     
            # stoploss_order = await place_stop_loss_order(config.TRADE_SYMBOL, 'sell', quantity, stop_price)           
            print('Stoploss created at:', stop_price)    
            stoplossFound = True                       
        # Check if there is a long position
        # elif position['side'] == 'Buy' and stoplossFound == True:   
        #     if stop_price == 0:
        #         stop_price = int(position['liquidationPrice'] + stop_distance)
        #     else:
        #         stop_price = stop_price + stop_distance
        #     print(f'Edit stop order')
        #     price = position['liquidationPrice'] * 0.0001
        #     params = {'stopLossEp': stop_price}
        #     # print('4')
        #     if difference in range(0,2):              
        #         time.sleep(3)
   
        #     exchange.edit_order(id=stoploss_order['id'], symbol=config.TRADE_SYMBOL, price=price,params=params)         
        # Update the stop-loss and limit orders
        elif position['side'] == 'Sell' and stoplossFound == True  and addStoploss == False:
            # Remove stop loss
            # print('8')
            exchange.cancel_order(stoploss_order['id'], symbol)                  
            print('Remove Stoploss')
            cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
            # print('All stoploss orders removed',cancelResponse)
            stoploss_order = None    

In [ ]:
async def stochStrategySell(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity):
    # check for stoploss conditions
    if latest_k > latest_d:
        print(f'latest_k {latest_k} > latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 80 <= latest_k <= 100 and 50 <= latest_d < 80:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 50 <= latest_k < 80 and 80 <= latest_d < 100:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 50 <= latest_k < 80 and 20 <= latest_d < 50:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 20 <= latest_k < 50 and 0 <= latest_d < 20:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
    elif latest_k < latest_d:
        print(f'latest_k {latest_k} < latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 50 <= latest_k < 80 and 80 <= latest_d <= 100:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 50 <= latest_k < 80 and 50 <= latest_d < 80:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 20 <= latest_k < 50 and 50 <= latest_d < 80:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 0 <= latest_k < 20 and 20 <= latest_d < 50:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
    else:
        print(f'latest_k {latest_k} == latest_d {latest_d}')

In [ ]:
async def stochStrategyBuy(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity):
    # check for stoploss conditions
    if latest_k > latest_d:
        print(f'latest_k {latest_k} > latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 80 <= latest_k <= 100 and 50 <= latest_d < 80:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 50 <= latest_k < 80 and 50 <= latest_d < 80:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 50 <= latest_k < 80 and 20 <= latest_d < 50:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 20 <= latest_k < 50 and 0 <= latest_d < 20:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
    elif latest_k < latest_d:
        print(f'latest_k {latest_k} < latest_d {latest_d}')
        if 80 <= latest_k <= 100 and 80 <= latest_d <= 100:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 50 <= latest_k < 80 and 80 <= latest_d <= 100:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 50 <= latest_k < 80 and 50 <= latest_d < 80:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 20 <= latest_k < 50 and 50 <= latest_d < 80:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
        elif 20 <= latest_k < 50 and 20 <= latest_d < 50:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 0 <= latest_k < 20 and 20 <= latest_d < 50:
            # remove stoploss
            print("Removing stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=False)
        elif 0 <= latest_k < 20 and 0 <= latest_d < 20:
            # add stoploss
            print("Adding stoploss...")
            await placeStoplossOrder(lastMbtcTick,lastClose,difference,position,quantity,addStoploss=True)
    else:
        print(f'latest_k {latest_k} == latest_d {latest_d}')

In [ ]:
async def strategy(lastMbtcTick,lastClose,psar_indicator,lastRSO,latest_k,latest_d):
    global stop_price
  
    try:
        position = getPositionInfo()[0]

        quantity = position['amount']  
        difference = abs(int(lastMbtcTick) - int(lastClose))                                           
        if position['side'] == 'Sell':
            await stochStrategySell(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity)
        elif position['side'] == 'Buy':
            await stochStrategyBuy(latest_k,latest_d,lastMbtcTick,lastClose,difference,position,quantity)        
        # print(f'position: {position}')
    except Exception as e:
        print('Error {}'.format(e))
        pass
        
    # if stoploss_order != None:      
    #     # Always Remove stop loss and start over
    #     print('5')
    #     exchange.cancel_order(stoploss_order['id'], symbol)                  
    #     print('Remove Stoploss')
    #     cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
    #     print('All stoploss orders removed',cancelResponse)
    #     stoploss_order = None 
    # print(f'psar_indicator: {psar_indicator} lastClose: {lastClose}') 
    # if position['side'] == 'Sell' and psar_indicator > lastClose:
    #     time.sleep(7)  
    # if position['side'] == 'Buy' and psar_indicator < lastClose:
    #     time.sleep(7) 
    # print(f'lastRSO: {lastRSO} lastClose: {lastClose}') 
 

In [ ]:
def handle_kline(kline):
    # Sort kline data by timestamp in ascending order
    kline = sorted(kline, key=lambda x: x[0])

    # Print sorted kline data
    return kline

In [ ]:
klines = []
ticks = []
grouped_data = {}

async def send_message(message):

    global ohlcvData, tradesPrices, klineData, klines
    type = None
    if message != None:
        # print(message)
        if 'kline' in message:
            for kline in handle_kline(json.loads(message)['kline']):
                klines.append(kline)  
                # klines.append(kline)   
            # type = json.loads(message)['type']
 
            # print(len(klines))
            # for item in klines:
                
            #     timestamp = datetime.fromtimestamp(datetime.now().timestamp()).strftime("%Y-%m-%d %H:%M:%S")
            #     if timestamp not in grouped_data:
            #         grouped_data[timestamp] = []
                
            #     grouped_data[timestamp].append(item)
        # print('lengte kline: ',len(klines))
        if 'tick' in message:
            ticks.append(json.loads(message)['tick'])  
            if len(klines) > 30:    
                klineData = pd.DataFrame(klines,columns=['timestamp', 'interval', 'lastCloseEp', 'openEp', 'highEp', 'lowEp', 'closeEp', 'volume', 'turnoverEv'])
                lastMbtcTick = int(ticks[-1]['last'] * 0.0001) 
                lastClose = int(klines[-1][6] * 0.0001)
                
                # # Filter klineData to only include rows with a new minute timestamp
                klineData['timestamp'] = pd.to_datetime(klineData['timestamp'], unit='s')
                newMinuteMask = klineData['timestamp'].diff().dt.seconds.gt(0)
                klineData = klineData.loc[newMinuteMask]
                # Filter klineData to only include rows with 7-minute, 1-hour, or 1-day intervals
                # klineData['timestamp'] = pd.to_datetime(klineData['timestamp'], unit='ms')
                # klineData.set_index('timestamp', inplace=True)
                # klineData_1min = klineData.resample('1T').last().dropna()
                # klineData_7min = klineData.resample('7T').last().dropna()
                # klineData_1h = klineData.resample('1H').last().dropna()
                # klineData_1d = klineData.resample('1D').last().dropna()
                # klineData = pd.concat([klineData_1min, klineData_7min, klineData_1h, klineData_1d])
                # klineData.reset_index(inplace=True)

                # Calculate the Parabolic SAR
                psar_indicator = ta.trend.PSARIndicator(high=klineData['highEp'], low=klineData['lowEp'], close=klineData['closeEp'])
                klineData['sar'] = psar_indicator.psar()
                lastSar = int(klineData.iloc[-1]['sar'] * 0.0001)


                # sma = talib.SMA(klineData['closeEp'], timeperiod=14)
                # latest = stream.SMA(klineData['closeEp'], timeperiod=14)
                # assert (sma.iloc[-1] - latest) < 0.00001

                # print(sma.iloc[-1], latest)#1.6180066666666686 1.6180066666666668
                # display(klineData['timestamp'].head(14))
                # display(klineData['timestamp'].tail(14))
                # fastk, fastd = talib.STOCHRSI(klineData['closeEp'], timeperiod=14, fastk_period=14, fastd_period=3, fastd_matype=3)
                # # f, fd = stream.STOCHRSI(klineData['closeEp'], timeperiod=14, fastk_period=14, fastd_period=3, fastd_matype=3)
                # # print(fastk.iloc[-1], f)
                # # print(fastd.iloc[-1], fd)

                # latest_k = fastk.iloc[-1]
                # latest_d = fastd.iloc[-1]
                # print(f'latest_k {latest_k} latest_d {latest_d}')
                # # print(f'Stream latest_k {f} latest_d {fd}')                
                # # assert (fastk.iloc[-1] - f) < 5#64.32089013974793 59.52628987038199

                # # print(fastk.iloc[-1], f)
                # # print(fastd.iloc[-1], fd)

                # print(f'Assert latest_k {latest_k} latest_d {latest_d}')
                # # print(f'Stream latest_k {f} latest_d {fd}')
                # time.sleep(10)
                

                # Calculate the Stochastic RSI values
                k_period = 14
                d_period = 3
                rsi_period = 14
                # k_values = klineData['closeEp'].tail(k_period)
                # rsi_values = klineData['closeEp'].tail(rsi_period)
                # # Calculate the Stochastic RSI indicator
                # # rsi = talib.RSI(k_values, timeperiod=rsi_period)
                # fastk, slowk, slowd = talib.STOCHRSI(klineData['closeEp'], k_period, k_period, d_period, d_period)
                # print(f'fastk {fastk}, slowk {slowk}, slowd {slowd}')
                # latest_k = fastk[-1]
                # latest_d = slowd[-1]
                fastk_period = 14
                slowk_matype = 0
                slowk_period = 3
                slowd_period = 3
                slowd_matype = 0
                rsi1 = talib.RSI(klineData['closeEp'], rsi_period)
                # print(rsi1)
                k,d = talib.STOCH(rsi1, rsi1, rsi1, fastk_period=fastk_period,slowk_matype=slowk_matype,slowk_period=slowk_period,slowd_period=slowd_period,slowd_matype=slowd_matype)
                # print('k-1',k.iloc[-1])
                # print('d-1',d.iloc[-1])
                d = talib.SMA(k, slowd_period)
                # print('sma d-1',d.iloc[-1])
                # Calculate the RSO Stochastic
                # rso_indicator = ta.momentum.StochRSIIndicator(close=klineData['closeEp'], window=14, smooth1=3, smooth2=3)
                # klineData['rso_stoch_k'] = rso_indicator.stochrsi_k()
                # klineData['rso_stoch_d'] = rso_indicator.stochrsi_d()
                latest_k = k.iloc[-1]
                latest_d = d.iloc[-1]  
                # print(f'latest_k {latest_k} latest_d {latest_d}')            
                # klineData['rso_stoch'] = rso_indicator.stochrsi()
                # lastRSO = float(klineData.iloc[-1]['rso_stoch'] * 100)             
                # print(f'lastMbtcTick: {lastMbtcTick} lastClose: {lastClose}')
                await strategy(lastMbtcTick,lastClose,lastSar,0,latest_k,latest_d)
                # await mbtcStrategy(lastMbtcTick,secondLastMbtcTick,lastClose,secondLastClose)
        # if len(klines) > 100:
        #     await sarStrategy(klines)       
        for clientId in clients:
            client = clients[clientId]
            if len(klines) > 0:
                if client['status'] != 'new':
                    sendKlines = klines
                sendMessage = json.dumps({'status':client['status'],'type':json.loads(message)['type'],'klines': klines,'ticks':ticks})
                await client['websocket'].send(sendMessage)

In [ ]:
# connect to Phemes websocket server
async def connectPhemexWS():
    global interval
    async for websocket in websockets.connect(config.WSSE_PHEMEX_TESTNET):
        try:
            print('Phemex Websocket Server is connected!')
            subscribe_ticker_msg = json.dumps({
                "id": 0,
                "method": "tick.subscribe",
                "params": [config.MBTC_SYMBOL]
            })
            await websocket.send(subscribe_ticker_msg)

            subscribe_kline_msg = json.dumps({
                "id": 0,
                "method": "kline.subscribe",
                "params": [config.TRADE_SYMBOL_BTCUSD,60]
            })
            await websocket.send(subscribe_kline_msg)

            while True:
                message = await websocket.recv()            
                # if 'book' in message:
                #     print(message)
                await send_message(message)
        except websockets.ConnectionClosed:
            continue

In [ ]:
async def register(websocket):
    # Assign a unique ID to the client
    client_id = len(clients) + 1
    clients[client_id] = {"websocket":websocket,"status":"new","info":{}}
    # await websocket.send(f"You are client {client_id}")
    print(f"Client {client_id} connected")
    return client_id

async def unregister(client_id):
    # Remove the client from the clients array
    del clients[client_id]
    print(f"Client {client_id} disconnected")

async def handle_message(client_id, message):
    # Handle incoming messages from the client
    print(f"Received message from client {client_id}: {json.loads(message)['status']}")
    # time.sleep(2000)
    # Store extra client information in clients array
    clients[client_id]["status"] = json.loads(message)['status']

async def handle_client(websocket, path):
    client_id = await register(websocket)
    try:
        async for message in websocket:
            await handle_message(client_id, message)
    except websockets.exceptions.ConnectionClosed:
        pass
    finally:
        # Unregister the client when they disconnect
        await unregister(client_id)

In [ ]:
# start python localhost websocket server
async def start_server():
    try:
        async with websockets.serve(handle_client, host="localhost", port=8765):
            print("Server started")
            await connectPhemexWS()
            await asyncio.Future()  # Keep the server running
    except Exception as e:
        print('Error {}'.format(e))
        pass

In [ ]:
if __name__ == '__main__':
    asyncio.run(start_server())